# Създаване на AI агенти с постоянна памет чрез Cognee

Този тетрадка демонстрира как да изградите интелигентни AI агенти със сложни възможности за памет, използвайки [**cognee**](https://www.cognee.ai/) - отворен код за AI памет, който комбинира графи на знания, семантично търсене и управление на сесии за създаване на контекстуално осведомени AI системи.

## 🎯 Цели на обучението

До края на този урок ще разберете как да:
- **Създавате графи на знания, подкрепени от вграждания**: Превръщате неструктуриран текст в структуриран, запитваем знание
- **Прилагате памет за сесии**: Създавате разговори с множество завои с автоматично запазване на контекста
- **Запазвате разговори**: По желание съхранявате важни взаимодействия в дългосрочна памет за бъдещи справки
- **Запитвате чрез естествен език**: Достъпвате и използвате исторически контекст в нови разговори
- **Визуализирате паметта**: Изследвате връзките в графа на знанията на вашия агент


## 🏗️ Какво ще създадете

В този урок ще създадем **Асистент за програмиране** с постоянна памет, който:

### 1. **Създаване на база знания**
   - Въвежда информация за профила и експертизата на разработчика
   - Обработва принципи и добри практики за програмиране на Python
   - Съхранява исторически разговори между разработчици и AI асистенти

### 2. **Разговори с осъзнаване на сесията**
   - Поддържа контекст през множество въпроси в една и съща сесия
   - Автоматично кешира всяка двойка въпрос/отговор за ефективно извличане
   - Осигурява последователни и контекстуални отговори въз основа на историята на разговора

### 3. **Дългосрочна памет**
   - Запазва важни разговори в дългосрочна памет
   - Извлича релевантни спомени от базата знания и минали сесии, за да информира нови взаимодействия
   - Изгражда растяща база знания, която се подобрява с времето

### 4. **Интелигентно извличане на памет**
   - Използва семантично търсене, базирано на графи, за намиране на релевантна информация в цялата съхранена база знания
   - Филтрира търсенията по подгрупи данни (информация за разработчици срещу принципи)
   - Комбинира множество източници на данни, за да предостави изчерпателни отговори


## 📋 Предварителни изисквания и настройка

### Системни изисквания

Преди да започнете, уверете се, че разполагате с:

1. **Python среда**
   - Python 3.9 или по-нова версия
   - Виртуална среда (препоръчително)
   
2. **Redis кеш** (Необходим за управление на сесиите)
   - Локален Redis: `docker run -d -p 6379:6379 redis`
   - Или използвайте управлявана услуга за Redis
   
3. **Достъп до LLM API**
   - OpenAI API ключ или други доставчици (вижте [документацията](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Конфигурация на база данни**
   - По подразбиране не се изисква конфигурация. Cognee използва бази данни, базирани на файлове (LanceDB и Kuzu)
   - По желание можете да настроите Azure AI Search като векторно хранилище (вижте [документацията](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Конфигурация на средата

Създайте файл `.env` в директорията на вашия проект със следните променливи:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Разбиране на архитектурата на паметта на Cognee

### Как работи Cognee

Cognee предоставя усъвършенствана система за памет, която надхвърля простото съхранение на ключ-стойност:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Основни компоненти:

1. **Граф на знанията**: Съхранява обекти, връзки и семантични връзки
2. **Векторни вграждания**: Позволява семантично търсене във всички съхранени данни
3. **Кеш на сесията**: Поддържа контекста на разговора в рамките на и между сесиите
4. **NodeSets**: Организират данните в логически категории за целенасочено извличане

### Видове памет в този урок:

- **Постоянна памет**: Дългосрочно съхранение в графа на знанията
- **Памет на сесията**: Временен контекст на разговора в Redis кеш
- **Семантична памет**: Търсене на сходство на база вектори във всички данни


## 📦 Инсталиране на необходимите пакети

Инсталирайте Cognee с поддръжка на Redis за управление на сесиите:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Инициализиране на среда и зареждане на библиотеки

Уверете се, че:
1. Redis работи (например чрез Docker: `docker run -d -p 6379:6379 redis`)
2. Променливите на средата са зададени преди импортиране на модулите за кеширане
3. Ако е необходимо, рестартирайте ядрото и изпълнете клетките по ред

Следващата клетка ще:
1. Зареди променливите на средата от `.env`
2. Конфигурира Cognee с вашите настройки за LLM
3. Активира кеширане за управление на сесии
4. Провери дали всички компоненти са правилно свързани


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Конфигуриране на директории за съхранение

Cognee използва две отделни директории за своите операции:
- **Корен на данните**: Съхранява въведените документи и обработените данни
- **Системен корен**: Съдържа базата данни на графа на знанието и системните метаданни

Ще създадем изолирани директории за този урок, както следва:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Нулиране на състоянието на паметта

Преди да започнем да изграждаме нашата система за памет, нека се уверим, че започваме на чисто.

> 💡 **Съвет**: Можете да пропуснете тази стъпка, ако искате да запазите съществуващите спомени от предишните си изпълнения, когато използвате този тефтер по-късно.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Част 1: Създаване на база знания

### Източници на данни за нашия асистент за разработчици

Ще използваме три типа данни, за да създадем изчерпателна база знания:

1. **Профил на разработчика**: Лична експертиза и технически опит
2. **Най-добри практики в Python**: Принципите на Python с практически насоки
3. **Исторически разговори**: Минали сесии въпроси и отговори между разработчици и AI асистенти

Тези разнообразни данни позволяват на нашия агент да:
- Разбира техническия контекст на потребителя
- Прилага най-добрите практики в препоръките
- Учи от предишни успешни взаимодействия


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Преобразуване на данни в граф на знания

Сега ще преобразуваме нашия необработен текст в структурирана памет. Този процес:

1. **Добавя данни към NodeSets**: Организира информацията в логически категории
   - `developer_data`: Профил на разработчика и разговори
   - `principles_data`: Най-добри практики и насоки за Python

2. **Изпълнява Cognify Pipeline**: Извлича обекти, взаимоотношения и създава вграждания
   - Идентифицира ключови концепции
   - Създава семантични връзки между свързана информация
   - Генерира векторни вграждания

Това може да отнеме няколко момента, докато LLM обработва текста и изгражда структурата на графа:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Визуализирайте графа на знанията

Нека разгледаме структурата на нашия граф на знанията. Визуализацията показва:
- **Възли**: Елементи, извлечени от текста (концепции, технологии, хора)
- **Ръбове**: Връзки и свързаности между елементите
- **Клъстери**: Свързани концепции, групирани по семантична прилика

Отворете генерирания HTML файл в браузъра си, за да изследвате графа интерактивно:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Обогатете паметта с Memify

Функцията `memify()` анализира графа на знанията и генерира интелигентни правила за данните. Този процес:
- Идентифицира модели и добри практики
- Създава приложими насоки въз основа на съдържанието
- Установява връзки между различни области на знания

Тези правила помагат на агента да взема по-информирани решения при отговаряне на въпроси. Заснемането на вторична визуализация ви позволява да сравните как графът се уплътнява след обогатяването.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Част 2: Интелигентно извличане на памет

### Демонстрация 1: Интеграция на знания от различни документи

Сега, когато нашият граф на знания е изграден, нека тестваме как Cognee комбинира информация от множество източници, за да отговаря на сложни въпроси.

Първото запитване демонстрира:
- **Семантично разбиране**: Намиране на релевантни концепции, дори когато не са изрично споменати
- **Кръстосано препращане**: Комбиниране на профил на разработчик с принципи на Python
- **Контекстуално разсъждение**: Прилагане на добри практики към конкретни реализации

### Демонстрация 2: Филтрирано търсене с NodeSets

Второто запитване показва как да се насочите към специфични подмножества на графа на знания:
- Използва параметъра `node_name`, за да търси само в `principles_data`
- Осигурява фокусирани отговори от конкретна област на знания
- Полезно, когато ви е необходима информация, специфична за дадена област


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Част 3: Настройка на управление на сесията

### Активиране на памет за разговори

Управлението на сесията е от съществено значение за поддържане на контекста през множество взаимодействия. Тук ще:

1. **Инициализираме потребителски контекст**: Създадем или извлечем потребителски профил за проследяване на сесията
2. **Конфигурираме кеширащ механизъм**: Свържем се с Redis за съхранение на историята на разговорите
3. **Активираме сесийни променливи**: Настроим контекстуални променливи, които се запазват между заявките

> ⚠️ **Важно**: Това изисква Redis да бъде стартиран и `CACHING=true` в средата ви


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Помощна функция: Преглед на историята на сесията

Тази помощна функция ни позволява да разгледаме историята на разговорите, съхранена в Redis. Полезна е за:
- Отстраняване на проблеми с управлението на сесии
- Проверка дали разговорите се кешират
- Разбиране какъв контекст е достъпен за агента


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Сесия 1: Лаборатория за асинхронна поддръжка — Първи въпрос

Започнете сесията `async-support-lab`, като попитате за подходящи за телеметрия asyncio модели за масивен уеб скрейпър. Графът вече има информация за asyncio, aiohttp и практики за мониторинг, така че отговорът трябва да отразява предишни разговори, като същевременно се адаптира към новия въпрос.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Проверка на паметта на сесия 1 след първия обмен

Изпълнението на `show_history(session_1)` веднага след първоначалния въпрос потвърждава, че Cognee е записал както въпроса, така и отговора в Redis. Трябва да видите един запис с насоки за едновременност.


In [ ]:
await show_history(session_1)

## Сесия 1: Последващи действия относно моделите данни

След това питаме: "Кога трябва да избера dataclasses вместо Pydantic?" използвайки същия идентификатор на сесията. Cognee трябва да обедини принципите на Python с предишните разговори за FastAPI, за да предостави детайлен съвет—демонстрирайки, че контекстът се пренася в рамките на именувана сесия.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Потвърдете, че историята на Сесия 1 съдържа и двата хода

Още едно извикване на `show_history(session_1)` трябва да покаже две записи за въпроси и отговори. Това съответства на стъпката "възпроизвеждане на паметта" от лабораторията Mem0 и доказва, че допълнителните ходове разширяват същия транскрипт.


In [ ]:
await show_history(session_1)

## Сесия 2: Дискусия за преглед на дизайна — Нова сесия

За да покажем изолация между дискусиите, стартираме `design-review-session` и молим за насоки за записване на прегледи на инциденти. Въпреки че основната база знания е същата, новият идентификатор на сесията поддържа стенограмите отделни.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Преглед на Сесия 2 История

`show_history(session_2)` трябва да показва само двойката подкана/отговор за преглед на дизайна. Сравнете го със Сесия 1, за да подчертаете как Cognee поддържа независими преписи, докато използва споделената графика на знания.


In [ ]:
await show_history(session_2)

## Обобщение

Поздравления! Току-що добавихте на вашия асистент за кодиране истински слой за дългосрочна памет, задвижван от Cognee.

В този урок взехте сурово съдържание за разработчици (код, документация, чатове) и го превърнахте в граф + векторна памет, която вашият агент може да търси, анализира и непрекъснато подобрява.

Какво научихте:

1. **От суров текст до AI памет**: Как Cognee обработва неструктурирани данни и ги превръща в интелигентна, търсима памет, използвайки комбинирана архитектура от вектор + граф на знания.

2. **Обогатяване на графа с memify**: Как да надхвърлите основното създаване на граф и да използвате memify, за да добавите извлечени факти и по-богати връзки върху съществуващия си граф.

3. **Множество стратегии за търсене**: Как да извършвате заявки към паметта с различни видове търсене (въпроси и отговори, осъзнати за графа, завършване в стил RAG, прозрения, сурови фрагменти, търсене на код и др.) в зависимост от нуждите на вашия агент.

4. **Визуално изследване**: Как да инспектирате и отстранявате грешки в това, което Cognee е създала, използвайки визуализации на графа и интерфейса на Cognee, за да видите как е структурирано знанието.

5. **Памет, осъзната за сесията**: Как да комбинирате контекста на всяка сесия с постоянна семантична памет, така че агентите да могат да запомнят между различни изпълнения, без да изтича информация между потребителите.


## Основни изводи
1. Памет като граф на знания, подкрепен от вграждания

    - **Структурирано разбиране**: Cognee комбинира хранилище за вектори и графово хранилище, така че вашите данни да са както търсими по смисъл, така и свързани чрез взаимоотношения. По подразбиране Cognee използва бази данни, базирани на файлове (LanceDB за вектори, Kuzu за графова база данни).

    - **Търсене, осъзнато за взаимоотношения**: Отговорите могат да бъдат основани не само на „подобен текст“, но и на това как се свързват обектите.

    - **Жива памет**: Слоят памет се развива, расте и остава достъпен за заявки като един свързан граф. 

2. Режими на търсене и разсъждение
    - **Хибридно извличане**: Търсенето комбинира векторна прилика, графова структура и разсъждения на LLM, от директно извличане на части до въпроси, осъзнати за графа.

    - **Приспособяване на режима към задачата**: Използвайте режими в стил завършване, когато искате отговори на естествен език, и режими за части/резюме/граф, когато вашият агент се нуждае от суров контекст или за да управлява собствените си разсъждения.

3. Персонализирани, осъзнати за сесия агенти
    - **Контекст на сесията + дългосрочна памет**: Cognee държи краткосрочния контекст на „нишката“ отделно от дългосрочната памет на потребителя или организацията. 

## Приложения в реалния свят

1. **Вертикални AI агенти**

    Използвайте модела от този бележник, за да захраните интелигентни за домейна копилоти, които се базират на Cognee като основа за извличане и разсъждение:

- **Копилоти за разработчици**: Асистенти за преглед на код, анализ на инциденти и архитектура, които преминават през код, API, проектна документация и билети като единен граф на паметта.

- **Копилоти за клиенти**: Асистенти за поддръжка или успех, които извличат информация от продуктова документация, често задавани въпроси, бележки от CRM и минали билети с осъзнато за графа извличане и цитирани отговори.

- **Вътрешни експертни копилоти**: Асистенти за политика, правни или сигурност, които разсъждават върху взаимосвързани правила, насоки и исторически решения, вместо върху изолирани PDF файлове.

    Cognee е изрично позициониран като постоянна, точна памет за AI агенти, предоставяйки жив граф на знания, който се интегрира зад вашия агент и заменя случайни комбинации от хранилища за вектори и персонализиран графов код. 

2. **Обединяване на изолирани данни в една памет**

    Същият подход също така ви помага да изградите единен слой памет от разпръснати източници:

- **От изолирани данни към един граф**: Въвеждайте структурирани (напр. бази данни) и неструктурирани данни (напр. документи, чатове) в един граф, подкрепен от вграждания, вместо отделни индекси за всяка система. 

- **Разсъждение между източници с цитати**: Изпълнявайте многоетапно разсъждение върху всичко – „свързвайте“ логове, метрики и документи чрез графа – и все пак връщайте обосновани отговори с произход. 

- **Центрове за знания**: За домейни като банкиране или образование, Cognee вече се използва за обединяване на PDF файлове, вътрешни системи и данни от приложения в един граф на знания с вектори, така че агентите да могат да отговарят на въпроси с прецизен, цитиран контекст. 

## Следващи стъпки

Вие сте внедрили основния цикъл на паметта. Ето естествени разширения, които можете да опитате сами (вижте [документацията на Cognee](https://docs.cognee.ai/) за подробности):

1. **Експериментирайте с времева осъзнатост**: Включете времевата функция на Cognee, за да извличате събития и времеви отпечатъци от текст.

2. **Въведете разсъждение, базирано на онтология**: Дефинирайте OWL онтология за вашия домейн. Използвайте поддръжката на онтология на Cognee, за да се основават извлечените обекти и взаимоотношения на тази схема, подобрявайки качеството на графа и отговорите, специфични за домейна. 

3. **Добавете обратна връзка**: Позволете на Cognee да коригира теглата на ръбовете на графа въз основа на реална обратна връзка от потребителите, така че извличането да се подобрява с времето, вместо да остава статично. 

4. **Оптимизирайте за персонализация и поведение на сесията**: Използвайте потребителски идентификатори, наематели и набори от данни, за да предоставите на всеки човек или екип собствен изглед върху споделения двигател на паметта. 

5. **Разширете към по-сложни агенти**: Интегрирайте Cognee в рамки за агенти, за да изградите системи с множество агенти, които всички споделят един и същ слой памет. *Microsoft Agent Framework x Cognee plugin скоро ще бъде наличен.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за каквито и да е недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
